In [1]:
import os
import glob
import pandas as pd
import numpy as np
import numpy.matlib
import scipy as sc
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, QuantileTransformer
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from torch.utils.data import DataLoader

from utils import create_save_folder, EarlyStopping

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

out_train = out_train.fillna(out_train.mean())
out_train.head()

stock_id,0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,50,51,52,53,55,56,58,59,60,61,62,63,64,66,67,68,69,70,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,93,94,95,96,97,98,99,100,101,102,103,104,105,107,108,109,110,111,112,113,114,115,116,118,119,120,122,123,124,125,126
time_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,0.004136,0.006340,0.001848,0.005300,0.004468,0.006234,0.007651,0.003624,0.010036,0.007291,0.005707,0.005918,0.002148,0.002627,0.005091,0.005485,0.004615,0.016663,0.002883,0.002915,0.002218,0.006377,0.006338,0.004405,0.008170,0.003655,0.002963,0.003233,0.004113,0.002649,0.006633,0.002586,0.003092,0.004755,0.008078,0.008372,0.002707,0.004864,0.002184,0.004094,0.002090,0.004551,0.001848,0.002206,0.003171,0.003947,0.002345,0.004885,0.002594,0.007041,0.005736,0.003274,0.004072,0.003928,0.004460,0.008681,0.004841,0.001808,0.004759,0.002675,0.002242,0.002625,0.002841,0.005403,0.005920,0.002803,0.005253,0.003856,0.002477,0.006344,0.007048,0.005846,0.003187,0.006131,0.005706,0.003294,0.003858,0.006019,0.004970,0.003590,0.008780,0.002126,0.005341,0.004152,0.005130,0.006254,0.003832,0.001742,0.004580,0.006088,0.005514,0.005007,0.003218,0.002995,0.004273,0.003129,0.004710,0.007743,0.002893,0.005967,0.006204,0.003708,0.004267,0.007944,0.003336,0.002571,0.003035,0.004862,0.002942,0.004112,0.001919,0.008067
11,0.001445,0.002099,0.000806,0.002774,0.001852,0.002562,0.004670,0.002458,0.002291,0.002529,0.002352,0.001634,0.000640,0.001072,0.001826,0.003172,0.002474,0.003902,0.003411,0.001601,0.001622,0.002445,0.002464,0.001417,0.003665,0.001991,0.000663,0.001917,0.000956,0.000688,0.007245,0.000924,0.001146,0.001555,0.004299,0.002344,0.001121,0.004665,0.000652,0.001330,0.000472,0.001861,0.000728,0.000760,0.001998,0.001296,0.001610,0.001958,0.001335,0.003729,0.002891,0.001223,0.001489,0.001607,0.001718,0.003743,0.001663,0.001230,0.001405,0.001449,0.001769,0.001133,0.002418,0.001837,0.002326,0.002262,0.003308,0.002284,0.001251,0.003299,0.004387,0.003544,0.001672,0.001918,0.002037,0.001285,0.001493,0.002496,0.001209,0.001609,0.004821,0.001003,0.002341,0.001560,0.002393,0.002656,0.001619,0.001013,0.002152,0.002691,0.001486,0.001788,0.001224,0.000965,0.001204,0.001339,0.002119,0.002275,0.001195,0.003635,0.001859,0.002443,0.001382,0.002469,0.002030,0.000839,0.001271,0.002095,0.001518,0.001891,0.001123,0.003965
16,0.002168,0.002456,0.001581,0.002986,0.002213,0.003253,0.004303,0.002178,0.001841,0.003299,0.002363,0.003923,0.001702,0.002114,0.002539,0.002653,0.002831,0.003806,0.002137,0.001902,0.001629,0.002742,0.002465,0.002162,0.006047,0.002152,0.001239,0.002344,0.002127,0.002431,0.003436,0.001612,0.002205,0.001976,0.004228,0.002748,0.001477,0.004131,0.001774,0.002267,0.001275,0.002703,0.001308,0.001800,0.002026,0.002777,0.001783,0.002092,0.001784,0.003094,0.002886,0.003132,0.002563,0.002423,0.002020,0.003401,0.001461,0.001461,0.002030,0.002063,0.002289,0.001413,0.002510,0.001670,0.002804,0.003364,0.003708,0.002256,0.002166,0.002735,0.002815,0.003735,0.002981,0.001996,0.002094,0.002147,0.001559,0.002654,0.002217,0.002112,0.002367,0.001688,0.001896,0.002378,0.003085,0.003712,0.003618,0.001366,0.002965,0.002639,0.003431,0.002939,0.002458,0.001799,0.001518,0.002630,0.002738,0.002752,0.001753,0.003406,0.002687,0.002704,0.001949,0.002195,0.003410,0.002569,0.002137,0.001893,0.002131,0.002428,0.001548,0.003161
31,0.002195,0.002807,0.001599,0.004437,0.002256,0.003072,0.005401,0.002149,0.003997,0.003696,0.002341,0.003581,0.001335,0.001802,0.002007,0.001897,0.002201,0.005206,0.001827,0.002071,0.001471,0.002550,0.002127,0.001638,0.004563,0.001634,0.000883,0.001073,0.003748,0.001881,0.002581,0.001369,0.002386,0.002128,0.002666,0.002927,0.001557,0.001945,0.000841,0.001757,0.000410,0.002066,0.001285,0.001337,0.001742,0.002524,0.001936,0.002696,0.001466,0.004116,0.003129,0.002059,0.002323,0.002589,

knn++を使ってkfoldの分割を行う

In [3]:
nfolds = 5
index = []
totDist = []
values = []

mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0] / nfolds)

mat = np.c_[mat, np.arange(mat.shape[0])]

lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
lineNumber = np.sort(lineNumber)[::-1]
lineNumber

array([2150, 1604, 1504, 1161,  196])

たぶんKNN++の実装

In [4]:
for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [5]:
with open('/home/yoshikawa/work/kaggle/OPVP/output/feature_model/20210831/1/train.pkl', 'rb') as f:
    df_train = pickle.load(f)

df_train['time_id'] = df_train['row_id'].apply(lambda x:x.split('-')[1])

col_names = list(df_train)
col_names.remove('target')
col_names.remove('row_id')
col_names.remove('stock_id')
col_names.remove('time_id')
len(col_names)

391

In [6]:
df_train.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []

for col in col_names:
    #正規分布になるよう非線形変換
    qt = QuantileTransformer(random_state=55, n_quantiles=2000, output_distribution='normal')
    df_train[col] = qt.fit_transform(df_train[[col]])
    qt_train.append(qt)


In [7]:
from sklearn.cluster import KMeans

train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()
ids = corr.index
corr

stock_id,0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,50,51,52,53,55,56,58,59,60,61,62,63,64,66,67,68,69,70,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,93,94,95,96,97,98,99,100,101,102,103,104,105,107,108,109,110,111,112,113,114,115,116,118,119,120,122,123,124,125,126
stock_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.798070,0.841460,0.647471,0.852778,0.853056,0.665597,0.835231,0.521143,0.703246,0.827460,0.812554,0.863764,0.863894,0.876982,0.863663,0.871740,0.718691,0.859322,0.883685,0.840467,0.839073,0.867195,0.862066,0.820183,0.851646,0.816843,0.872960,0.777400,0.853222,0.830139,0.860078,0.849721,0.841918,0.796506,0.790850,0.871583,0.815397,0.847223,0.867359,0.842520,0.829196,0.841014,0.857526,0.856540,0.760915,0.864465,0.862372,0.868446,0.801338,0.795418,0.777022,0.783066,0.832884,0.678273,0.796276,0.708355,0.849639,0.846981,0.873495,0.855358,0.800851,0.873719,0.844152,0.829740,0.806061,0.768177,0.799546,0.722904,0.775386,0.586150,0.371463,0.843755,0.767821,0.755401,0.834652,0.729698,0.822272,0.813602,0.758473,0.782248,0.873980,0.845500,0.852008,0.781913,0.735068,0.829811,0.795965,0.848455,0.776698,0.872269,0.846168,0.884945,0.873282,0.876094,0.815471,0.838547,0.749293,0.809180,0.847949,0.831716,0.859928,0.853943,0.822769,0.849980,0.857607,0.858696,0.814953,0.888719,0.759692,0.860547,0.807063
1,0.798070,1.000000,0.694983,0.707773,0.791688,0.801733,0.708319,0.750894,0.723175,0.778223,0.870090,0.856337,0.737631,0.746378,0.853675,0.816277,0.797698,0.750514,0.753604,0.781822,0.746561,0.835399,0.868211,0.838020,0.800788,0.786425,0.757869,0.766076,0.736677,0.760194,0.773059,0.739113,0.773433,0.815958,0.791589,0.717352,0.746052,0.733888,0.724482,0.799706,0.729033,0.841099,0.731454,0.764203,0.794225,0.755701,0.777661,0.796547,0.773748,0.813273,0.823197,0.619506,0.692212,0.785864,0.670692,0.885674,0.755661,0.733404,0.825583,0.786756,0.723756,0.709761,0.790906,0.832011,0.815274,0.778433,0.739308,0.750984,0.605533,0.821654,0.613757,0.551079,0.776212,0.772904,0.814806,0.766643,0.743791,0.881767,0.773897,0.761355,0.767034,0.757149,0.825323,0.854568,0.750874,0.787766,0.757831,0.680632,0.791129,0.846505,0.827992,0.752273,0.791175,0.784787,0.787419,0.741945,0.820834,0.748351,0.765209,0.829181,0.788534,0.749605,0.813933,0.839468,0.760083,0.741288,0.799673,0.845167,0.819863,0.753699,0.730829,0.880584
2,0.841460,0.694983,1.000000,0.534588,0.835739,0.813608,0.517432,0.885969,0.415829,0.601032,0.803774,0.742594,0.907172,0.901920,0.861403,0.815477,0.882961,0.617397,0.878068,0.916552,0.838042,0.737445,0.819790,0.860156,0.751689,0.843008,0.838578,0.879038,0.751550,0.861598,0.802993,0.913243,0.883166,0.806170,0.704182,0.793808,0.925914,0.811085,0.919675,0.853328,0.911330,0.811872,0.906371,0.885590,0.809894,0.704730,0.902994,0.895369,0.897121,0.691240,0.725758,0.802230,0.809490,0.775260,0.623892,0.677090,0.619293,0.889239,0.811555,0.908010,0.909056,0.838568,0.892892,0.804442,0.759454,0.762123,0.689550,0.746987,0.784037,0.691044,0.523586,0.209831,0.812888,0.696696,0.666724,0.844822,0.662584,0.773581,0.789289,0.710400,0.724259,0.921355,0.784888,0.847984,0.727378,0.639994,0.773781,0.820672,0.836812,0.697574,0.832929,0.872391,0.882024,0.879077,0.867752,0.813547,0.800513,0.754208,0.849081,0.763753,0.835048,0.882552,0.841273,0.752961,0.851451,0.890355,0.826682,0.766011,0.890501,0.704816,0.943743,0.722100
3,0.647471,0.707773,0.534588,1.000000,0.650228,0.663158,0.769600,0.627592,0.593529,0.747987,0.703660,0.690211,0.588374,0.627096,0.637897,0.666459,0.681931,0.674556,0.606450,0.634802,0.597385,0.762068,0.711533,0.648869,0.646698,0.613126,0.703551,0.597575,0.638322,0.653777,0.631494,0.616078,0.633751,0.699179,0.653703,0.567812,0.562095,0.625539,0.587580,0.677529,0.586488,0.703025,0.598057,0.596322,0.730462,0.760630,0.594760,0.631192,0.670686,0.776269,0.797645,0.559162,0.595737,0.661648,0.

In [8]:
kmeans = KMeans(n_clusters = 7, random_state=55).fit(corr.values)
kmeans.labels_

array([0, 6, 3, 1, 0, 0, 1, 3, 5, 1, 0, 6, 3, 3, 0, 0, 0, 1, 3, 3, 3, 6,
       0, 0, 2, 0, 0, 3, 2, 3, 2, 3, 3, 0, 2, 2, 3, 2, 3, 0, 3, 0, 3, 3,
       0, 6, 3, 3, 3, 6, 6, 2, 2, 2, 1, 6, 1, 3, 0, 3, 3, 0, 3, 0, 6, 2,
       2, 6, 2, 6, 5, 4, 2, 2, 6, 0, 1, 6, 2, 2, 2, 3, 0, 0, 6, 1, 2, 2,
       0, 6, 0, 3, 3, 3, 3, 0, 0, 2, 0, 6, 0, 3, 0, 6, 0, 3, 0, 6, 3, 6,
       3, 6], dtype=int32)

同じクラスで値の平均を取る。

In [9]:
l = []
for n in range(7):
    l.append([(x-1) for x in ((ids + 1) * (kmeans.labels_ == n)) if x > 0])
    
mat = []

for n, ind in enumerate(l):
    print(ind)
    df_new = df_train.loc[df_train['stock_id'].isin(ind)]
    df_new = df_new.groupby(['time_id']).agg(np.nanmean)
    df_new.loc[:, 'stock_id'] = str(n) + 'c1'
    mat.append(df_new)

mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'], inplace=True)
mat1

[0, 4, 5, 10, 15, 16, 17, 23, 26, 28, 29, 36, 42, 44, 48, 66, 69, 72, 85, 94, 95, 100, 102, 108, 109, 111, 113, 115, 118, 120]
[3, 6, 9, 18, 61, 63, 86, 97]
[27, 31, 33, 37, 38, 40, 58, 59, 60, 74, 75, 77, 82, 83, 88, 89, 90, 98, 99, 110]
[2, 7, 13, 14, 19, 20, 21, 30, 32, 34, 35, 39, 41, 43, 46, 47, 51, 52, 53, 64, 67, 68, 70, 93, 103, 104, 105, 107, 114, 119, 123, 125]
[81]
[8, 80]
[1, 11, 22, 50, 55, 56, 62, 73, 76, 78, 84, 87, 96, 101, 112, 116, 122, 124, 126]


,time_id,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_500,log_return_mean_500,log_return_std_500,log_return_sum_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,log_return2_sum_500,wap_balance_mean_500,wap_balance_std_500,wap_balance_sum_500,price_spread_mean_500,price_spread_std_500,price_spread_sum_500,bid_spread_mean_500,bid_spread_std_500,bid_spread_sum_500,ask_spread_mean_500,ask_spread_std_500,ask_spread_sum_500,volume_imbalance_mean_500,volume_imbalance_std_500,volume_imbalance_sum_500,total_volume_mean_500,total_volume_std_500,total_volume_sum_500,wap_mean_500,wap_std_500,wap_sum_500,log_return_realized_volatility_400,log_return_mean_400,log_return_std_400,log_return_sum_400,log_return2_realized_volatility_400,log_return2_mean_400,log_return2_std_400,log_return2_sum_400,wap_balance_mean_400,wap_balance_std_400,wap_balance_sum_400,price_spread_mean_400,price_spread_std_400,price_spread_sum_400,bid_spread_mean_400,bid_spread_std_400,bid_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,ask_spread_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,volume_imbalance_sum_400,total_volume_mean_400,total_volume_std_400,total_volume_sum_400,wap_mean_400,wap_std_400,wap_sum_400,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_200,log_return_mean_200,log_return_std_200,log_return_sum_200,log_return2_realized_volatility_200,log_return2_mean_200,log_return2_std_200,log_return2_sum_200,wap_balance_mean_200,wap_balance_std_200,wap_balance_sum_200,price_spread_mean_200,price_spread_std_200,price_spread_sum_200,bid_spread_mean_200,bid_spread_std_200,bid_spread_sum_200,ask_spread_mean_200,ask_spread_std_200,ask_spread_sum_200,volume_imbalance_mean_200,volume_imbalance_std_200,volume_imbalance_sum_200,total_volume_mean_200,total_volume_std_200,total_volume_sum_200,wap_mean_200,wap_std_200,wap_sum_200,log_return_realized_volatility_100,log_return_mean_100,log_return_std_100,log_return_sum_100,...,trade_order_count_amax_100,trade_amount_mean_100,trade_amount_amax_100,trade_amount_amin_100,stock_id,stock_id_target_enc,log_return_realized_volatility_mean_stock,log_return_realized_volatility_std_stock,log_return_realized_volatility_max_stock,log_return_realized_volatility_min_stock,log_return2_realized_volatility_mean_stock,log_return2_realized_volatility_std_stock,log_return2_realized_volatility_max_stock,log_return2_realized_volatility_min_stock,log_return_realized_volatility_500_mean_stock,log_return_realized_volatility_500_std_stock,log_return_realized_volatility_500_max_stock,log_return_realized_volatility_500_min_stock,log_return2_realized_volatility_500_mean_stock,log_return2_realized_volatility_500_std_stock,log_return2_realized_volatility_500_max_stock,log_return2_realized_volatility_500_min_stock,log_return_realized_volatility_400_mean_stock,log_return_realized_volatility_400_std_stock,log_return_realized_volatility_400_max_stock,log_return_r

In [10]:
mat = []
kmeans = []

In [11]:
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

In [12]:
mat1

,time_id,log_return_realized_volatility_0c1,log_return_realized_volatility_1c1,log_return_realized_volatility_2c1,log_return_realized_volatility_3c1,log_return_realized_volatility_4c1,log_return_realized_volatility_5c1,log_return_realized_volatility_6c1,log_return_mean_0c1,log_return_mean_1c1,log_return_mean_2c1,log_return_mean_3c1,log_return_mean_4c1,log_return_mean_5c1,log_return_mean_6c1,log_return_std_0c1,log_return_std_1c1,log_return_std_2c1,log_return_std_3c1,log_return_std_4c1,log_return_std_5c1,log_return_std_6c1,log_return_sum_0c1,log_return_sum_1c1,log_return_sum_2c1,log_return_sum_3c1,log_return_sum_4c1,log_return_sum_5c1,log_return_sum_6c1,log_return2_realized_volatility_0c1,log_return2_realized_volatility_1c1,log_return2_realized_volatility_2c1,log_return2_realized_volatility_3c1,log_return2_realized_volatility_4c1,log_return2_realized_volatility_5c1,log_return2_realized_volatility_6c1,log_return2_mean_0c1,log_return2_mean_1c1,log_return2_mean_2c1,log_return2_mean_3c1,log_return2_mean_4c1,log_return2_mean_5c1,log_return2_mean_6c1,log_return2_std_0c1,log_return2_std_1c1,log_return2_std_2c1,log_return2_std_3c1,log_return2_std_4c1,log_return2_std_5c1,log_return2_std_6c1,log_return2_sum_0c1,log_return2_sum_1c1,log_return2_sum_2c1,log_return2_sum_3c1,log_return2_sum_4c1,log_return2_sum_5c1,log_return2_sum_6c1,wap_balance_mean_0c1,wap_balance_mean_1c1,wap_balance_mean_2c1,wap_balance_mean_3c1,wap_balance_mean_4c1,wap_balance_mean_5c1,wap_balance_mean_6c1,wap_balance_std_0c1,wap_balance_std_1c1,wap_balance_std_2c1,wap_balance_std_3c1,wap_balance_std_4c1,wap_balance_std_5c1,wap_balance_std_6c1,wap_balance_sum_0c1,wap_balance_sum_1c1,wap_balance_sum_2c1,wap_balance_sum_3c1,wap_balance_sum_4c1,wap_balance_sum_5c1,wap_balance_sum_6c1,price_spread_mean_0c1,price_spread_mean_1c1,price_spread_mean_2c1,price_spread_mean_3c1,price_spread_mean_4c1,price_spread_mean_5c1,price_spread_mean_6c1,price_spread_std_0c1,price_spread_std_1c1,price_spread_std_2c1,price_spread_std_3c1,price_spread_std_4c1,price_spread_std_5c1,price_spread_std_6c1,price_spread_sum_0c1,price_spread_sum_1c1,price_spread_sum_2c1,price_spread_sum_3c1,price_spread_sum_4c1,price_spread_sum_5c1,price_spread_sum_6c1,bid_spread_mean_0c1,bid_spread_mean_1c1,bid_spread_mean_2c1,bid_spread_mean_3c1,bid_spread_mean_4c1,bid_spread_mean_5c1,bid_spread_mean_6c1,bid_spread_std_0c1,bid_spread_std_1c1,bid_spread_std_2c1,bid_spread_std_3c1,bid_spread_std_4c1,bid_spread_std_5c1,bid_spread_std_6c1,bid_spread_sum_0c1,bid_spread_sum_1c1,bid_spread_sum_2c1,bid_spread_sum_3c1,bid_spread_sum_4c1,bid_spread_sum_5c1,bid_spread_sum_6c1,ask_spread_mean_0c1,ask_spread_mean_1c1,ask_spread_mean_2c1,ask_spread_mean_3c1,ask_spread_mean_4c1,ask_spread_mean_5c1,ask_spread_mean_6c1,ask_spread_std_0c1,ask_spread_std_1c1,ask_spread_std_2c1,ask_spread_std_3c1,ask_spread_std_4c1,ask_spread_std_5c1,ask_spread_std_6c1,ask_spread_sum_0c1,ask_spread_sum_1c1,ask_spread_sum_2c1,ask_spread_sum_3c1,ask_spread_sum_4c1,ask_spread_sum_5c1,ask_spread_sum_6c1,volume_imbalance_mean_0c1,volume_imbalance_mean_1c1,volume_imbalance_mean_2c1,volume_imbalance_mean_3c1,volume_imbalance_mean_4c1,volume_imbalance_mean_5c1,volume_imbalance_mean_6c1,volume_imbalance_std_0c1,volume_imbalance_std_1c1,...,trade_log_return_realized_volatility_max_time_4c1,trade_log_return_realized_volatility_max_time_5c1,trade_log_return_realized_volatility_max_time_6c1,trade_log_return_realized_volatility_min_time_0c1,trade_log_return_realized_volatility_min_time_1c1,trade_log_return_realized_volatility_min_time_2c1,trade_log_return_realized_volatility_min_time_3c1,trade_log_return_realized_volatility_min_time_4c1,trade_log_return_realized_volatility_min_time_5c1,trade_log_return_realized_volatility_min_time_6c1,trade_log_return_realized_volatility_500_mean_time_0c1,trade_log_return_realized_volatility_500_mean_time_1c1,trade_log_return_realized_volatility_500_mean_time_2c1,trade_log_return_realized_volatility_500_mean_time_3c1,trade_log_return_r

In [13]:
nnn = ['time_id',
     'log_return_realized_volatility_0c1',
     'log_return_realized_volatility_1c1',     
     'log_return_realized_volatility_2c1',
     'log_return_realized_volatility_3c1',     
     'log_return_realized_volatility_6c1',
     'total_volume_mean_0c1',
     'total_volume_mean_1c1', 
     'total_volume_mean_2c1',
     'total_volume_mean_3c1', 
     'total_volume_mean_6c1',
     # 'trade_size_mean_0c1',
     # 'trade_size_mean_1c1', 
     # 'trade_size_mean_2c1',
     # 'trade_size_mean_3c1', 
     # 'trade_size_mean_6c1',
     'trade_order_count_mean_0c1',
     'trade_order_count_mean_1c1',
     'trade_order_count_mean_2c1',
     'trade_order_count_mean_3c1',
     'trade_order_count_mean_6c1',      
     'price_spread_mean_0c1',
     'price_spread_mean_1c1',
     'price_spread_mean_2c1',
     'price_spread_mean_3c1',
     'price_spread_mean_6c1',   
     'bid_spread_mean_0c1',
     'bid_spread_mean_1c1',
     'bid_spread_mean_2c1',
     'bid_spread_mean_3c1',
     'bid_spread_mean_6c1',       
     'ask_spread_mean_0c1',
     'ask_spread_mean_1c1',
     'ask_spread_mean_2c1',
     'ask_spread_mean_3c1',
     'ask_spread_mean_6c1',   
     'volume_imbalance_mean_0c1',
     'volume_imbalance_mean_1c1',
     'volume_imbalance_mean_2c1',
     'volume_imbalance_mean_3c1',
     'volume_imbalance_mean_6c1',       
     'price_spread_mean_0c1',
     'price_spread_mean_1c1',
     'price_spread_mean_2c1',
     'price_spread_mean_3c1',
     'price_spread_mean_6c1',
     # 'size_tau2_0c1',
     # 'size_tau2_1c1',
     # 'size_tau2_2c1',
     # 'size_tau2_3c1',
     # 'size_tau2_6c1'
     ] 

In [14]:
from utils import create_save_folder
df_train = pd.merge(df_train, mat1[nnn],how='left',on='time_id')
fm_path = create_save_folder('nb013')
pickle.dump(df_train, open(os.path.join(fm_path, "train.pkl"), 'wb'))

In [15]:
mat1= []
mat2= []

In [16]:
n_folds =5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=55)
scores = []

train_columns = list(df_train)
train_columns.remove('time_id')
train_columns.remove('row_id')
train_columns.remove('target')

for col in train_columns:
    df_train[col] = df_train[col].fillna(df_train[col].mean())
df_train.describe()

,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_sum,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_sum,wap_balance_mean,wap_balance_std,wap_balance_sum,price_spread_mean,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_std,ask_spread_sum,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_sum,total_volume_mean,total_volume_std,total_volume_sum,wap_mean,wap_std,wap_sum,log_return_realized_volatility_500,log_return_mean_500,log_return_std_500,log_return_sum_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,log_return2_sum_500,wap_balance_mean_500,wap_balance_std_500,wap_balance_sum_500,price_spread_mean_500,price_spread_std_500,price_spread_sum_500,bid_spread_mean_500,bid_spread_std_500,bid_spread_sum_500,ask_spread_mean_500,ask_spread_std_500,ask_spread_sum_500,volume_imbalance_mean_500,volume_imbalance_std_500,volume_imbalance_sum_500,total_volume_mean_500,total_volume_std_500,total_volume_sum_500,wap_mean_500,wap_std_500,wap_sum_500,log_return_realized_volatility_400,log_return_mean_400,log_return_std_400,log_return_sum_400,log_return2_realized_volatility_400,log_return2_mean_400,log_return2_std_400,log_return2_sum_400,wap_balance_mean_400,wap_balance_std_400,wap_balance_sum_400,price_spread_mean_400,price_spread_std_400,price_spread_sum_400,bid_spread_mean_400,bid_spread_std_400,bid_spread_sum_400,ask_spread_mean_400,ask_spread_std_400,ask_spread_sum_400,volume_imbalance_mean_400,volume_imbalance_std_400,volume_imbalance_sum_400,total_volume_mean_400,total_volume_std_400,total_volume_sum_400,wap_mean_400,wap_std_400,wap_sum_400,log_return_realized_volatility_300,log_return_mean_300,log_return_std_300,log_return_sum_300,log_return2_realized_volatility_300,log_return2_mean_300,log_return2_std_300,log_return2_sum_300,wap_balance_mean_300,wap_balance_std_300,wap_balance_sum_300,price_spread_mean_300,price_spread_std_300,price_spread_sum_300,bid_spread_mean_300,bid_spread_std_300,bid_spread_sum_300,ask_spread_mean_300,ask_spread_std_300,ask_spread_sum_300,volume_imbalance_mean_300,volume_imbalance_std_300,volume_imbalance_sum_300,total_volume_mean_300,total_volume_std_300,total_volume_sum_300,wap_mean_300,wap_std_300,wap_sum_300,log_return_realized_volatility_200,log_return_mean_200,log_return_std_200,log_return_sum_200,log_return2_realized_volatility_200,log_return2_mean_200,log_return2_std_200,log_return2_sum_200,wap_balance_mean_200,wap_balance_std_200,wap_balance_sum_200,price_spread_mean_200,price_spread_std_200,price_spread_sum_200,bid_spread_mean_200,bid_spread_std_200,bid_spread_sum_200,ask_spread_mean_200,ask_spread_std_200,ask_spread_sum_200,volume_imbalance_mean_200,volume_imbalance_std_200,volume_imbalance_sum_200,total_volume_mean_200,total_volume_std_200,total_volume_sum_200,wap_mean_200,wap_std_200,wap_sum_200,log_return_realized_volatility_100,log_return_mean_100,log_return_std_100,log_return_sum_100,...,log_return_realized_volatility_200_max_stock,log_return_realized_volatility_200_min_stock,log_return2_realized_volatility_200_mean_stock,log_return2_realized_volatility_200_std_stock,log_return2_realized_volatility_200_max_stock,log_return2_realized_volatility_200_min_stock,log_return_realized_volatility_100_mean_stock,log_return_realized_volatility_100_std_stock,log_return_realized_volatility_100_max_stock,log_return_realized_volatility_100_min_stock,log_return2_realized_volatility_100_mean_stock,log_return2_realized_volatility_100_std_stock,log_return2_realized_volatility_100_max_stock,log_return2_realized_volatility_100_min_stock,trade_log_return_realized_volatility_mean_stock,trade_log_return_realized_volatility_std_stock,trade_log_return_realized_volatility_max_stock,trade_log_return_realized_volatility_min_stock,trade_log_return_realized_volatility_500_mean_stock,trade_log_return_realized_volatility_500_std_stock,trade_log_return_realized_volatility_5

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def ToTensor(x):
    return torch.tensor(x.astype(np.float32)).to(device)
from torch.utils.data import Dataset

class DataSet(Dataset):
    def __init__(self, X, stock, y):
        self.X = X
        self.stock = stock
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.stock[index], self.y[index]

In [18]:
def rsmpe(y_true, y_pred):
    rsmpe = (torch.sqrt(torch.clamp(torch.mean(torch.square((y_true - y_pred) / y_true)),  min=1.0e-6)))
    assert rsmpe != 0
    return rsmpe

In [19]:
df_train.isnull().values.sum()

0

In [20]:
import subprocess
import shlex

def gpuinfo():
    """
    Returns size of total GPU RAM and used GPU RAM.

    Parameters
    ----------
    None

    Returns
    -------
    info : dict
        Total GPU RAM in integer for key 'total_MiB'.
        Used GPU RAM in integer for key 'used_MiB'.
    """

    command = 'nvidia-smi -q -d MEMORY | sed -n "/FB Memory Usage/,/Free/p" | sed -e "1d" -e "4d" -e "s/ MiB//g" | cut -d ":" -f 2 | cut -c2-'
    commands = [shlex.split(part) for part in command.split(' | ')]
    for i, cmd in enumerate(commands):
        if i==0:
            res = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        else:
            res = subprocess.Popen(cmd, stdin=res.stdout, stdout=subprocess.PIPE)

    l = res.communicate()[0].decode('utf-8').strip().split('\n')
    info = {'total_MiB':l[3], 'used_MiB':l[4]}
    return info

In [21]:
gpuinfo()

{'total_MiB': '16117', 'used_MiB': '12647'}

In [22]:
%load_ext autoreload
%autoreload
from model import FFNN

models = []
t_losses = [[] for i in range(n_folds)]
v_losses = [[] for i in range(n_folds)]
for fold in range(n_folds):
    indexes = np.arange(n_folds).astype(int)    
    indexes = np.delete(indexes,obj=fold, axis=0) 
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    indexes = [str(x) for x in indexes]
    values[fold] = [str(x) for x in values[fold]]

    X_train = df_train[df_train['time_id'].isin(indexes)]
    X_val = df_train.loc[df_train.time_id.isin(values[fold])]
    scaler = MinMaxScaler(feature_range=(-1, 1))

    num_data = X_train.drop(['stock_id', 'target', 'time_id', 'row_id'], axis=1)
    num_data = scaler.fit_transform(num_data)
    stock = X_train[['stock_id']].values
    y = X_train[['target']].values

    num_data_val = X_val.drop(['stock_id', 'target', 'time_id', 'row_id'], axis=1)
    num_data_val = scaler.transform(num_data_val) 
    stock_val = X_val[['stock_id']].values
    y_val = X_val[['target']].values
    
    train_dataset = DataSet(num_data, stock, y)
    train_loader = DataLoader(train_dataset, 4096, shuffle=True)
    
    model = FFNN(num_data.shape[1]).cuda()
    print(model)
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    earlystopping = EarlyStopping(patience=20, verbose=True, path=fm_path+'/checkpoint.pth')
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=7)
    
    epochs = 1000
    model.train()
    for i in range(epochs):
        train_loss, val_loss = 0, 0
        for j, (X, stock, y)in enumerate(train_loader):
            X, stock, y = X.to(torch.float).cuda(), stock.cuda(), y.to(torch.float).cuda()
            out = model(X, stock)
            loss = rsmpe(y, out)
            # print(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.shape[0]
        
        train_loss /= len(train_dataset)
        out = model(ToTensor(num_data_val), torch.tensor(stock_val).cuda())
        val_loss = rsmpe(ToTensor(y_val), out).item()
        
        print(f"{i+1} epoch - train_loss: {train_loss:.6f} val_loss: {val_loss:.6f}")
        # print(gpuinfo())
        t_losses[fold].append(train_loss)
        v_losses[fold].append(val_loss)
        earlystopping(val_loss, model)
        scheduler.step(val_loss)
        if earlystopping.early_stop:
            print("Early Stopping!!")
            break
    model.load_state_dict(torch.load(fm_path+'/checkpoint.pth'))
    models.append(model)


FFNN(
  (stock_embedding): Embedding(127, 24)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (hidden): Sequential(
    (0): Linear(in_features=455, out_features=128, bias=True)
    (1): SiLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): SiLU(inplace=True)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): SiLU(inplace=True)
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)
1 epoch - train_loss: 2.020089 val_loss: 0.254805
Validation loss decreased (inf --> 0.254805).  Saving model ...
2 epoch - train_loss: 0.299010 val_loss: 0.300019
EarlyStopping counter: 1 out of 20
3 epoch - train_loss: 0.303043 val_loss: 0.316029
EarlyStopping counter: 2 out of 20
4 epoch - train_loss: 0.286156 val_loss: 0.383631
EarlyStopping counter: 3 out of 20
5 epoch - train_loss: 0.282496 val_loss: 0.273083
EarlyStopping counter: 4 out of 20
6 epoch - train_loss: 0.284071 val_loss: 0.267777
EarlyStopping counter: 5 out of 20
7 epoch - tra

In [23]:
cv = 0
for model in models:
	model.eval()
	out = model(ToTensor(num_data_val), torch.tensor(stock_val).cuda())
	loss = rsmpe(ToTensor(y_val), out).item()
	cv += loss
	print(loss)
cv /= 5
cv

0.21894940733909607
0.2162133902311325
0.7457408905029297
0.20560182631015778
0.21142086386680603


0.3195852756500244

In [24]:
import matplotlib.pyplot as plt

for fold in range(n_folds):
	x = range(len(t_losses[fold]))
	x = [(i+1) * 5 for i in x]
	print(x)

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340, 345, 350, 355, 360, 365, 370, 375, 380, 385, 390, 395, 400, 405, 410, 415, 420, 425, 430, 435, 440, 445, 450, 455, 460, 465, 470, 475, 480, 485, 490, 495, 500, 505, 510]
[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215, 220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 300, 305, 310, 315, 320, 325, 330, 335, 340]
[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 21